In [ ]:
# This is a web crawler for the site spitogatos.gr. It is a property listing site in greece.

In [ ]:
import bs4 as bs
import requests
import pandas as pd
from urllib.parse import unquote

In [ ]:
# chage this link based on the area you want to select
url='https://www.spitogatos.gr/search/results/residential/rent/r100/m100m/'

In [ ]:
# Headers are used in order to "trick" the website that the user is not a bot
headers={}
headers['User-Agent']='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
link= unquote(url)
#    print(link)
text= requests.get(link, headers=headers).text
soup= bs.BeautifulSoup(text)

In [ ]:
# for this particular website the user needs to select the specified area he want in order for listings to show up
# Hypothetically we can make a list of all the links for the areas and loop through the entire website but I chose to do it in
# way so it does not take a long time to run


In [ ]:
# In this section the program take the specified number of listings for the particular area so the user only need to change 
# the initial link and the rest will be automated.
num_of_listing= soup.find('h2',class_="padding-left h5 searchTotalNumberOfResults").parent.find_next('b')
print(num_of_listing.text.strip())
lenght= int(float(num_of_listing.text.strip())*1000)

In [ ]:
# Makes a list of all the possible pages that this area has
page_links=[]
for number in range(0,lenght,10):
        #print(number)
        url_pages = url + 'offset_' + str(number)
        page_links.append(url_pages)1

In [ ]:
# Finds the links to all the posts and appends it into a dataframe. I used a dataframe because I ran through some error using lists
posts=[]
for pages in page_links:
    req1= unquote(pages)
    resp1 = requests.get(req1,headers=headers).text
    soup1 = bs.BeautifulSoup(resp1,'html.parser')
    for post in soup1.find_all('div',class_="bd padding-right"):
        post= post.a.get('href')
        posts.append(post)

df = pd.DataFrame(columns = ['link'],data=posts)

df.link.str.encode('utf-8')

In [ ]:
#This last part opens each post and scrapes the price, price per m^2, size in m^2, type of property, number of rooms, number of baths
# what floop the property is in, the parking availability, the year the property was built, the last update of the listing and 
# finally the specific area the property is in.

In [ ]:
prices=[]
pricesper=[]
sizes=[]
kinds=[]
heat_types=[]
num_of_rooms=[]
num_of_baths=[]
floors=[]
parking=[]
year_builts=[]
last_update=[]
area=[]
for i in range(len(df)):
    link = (df.link.iloc[i])
    link= unquote(link)
#    print(link)
    texts= requests.get(link, headers=headers).text
    soup2= bs.BeautifulSoup(texts)
    
    try:
        price= soup2.find('h6',string='Τιμή').parent.find_next('div') 
        prices.append(price.text.strip())
    except AttributeError:
        prices.append('None')
   
    try:
        priceper= soup2.find('h6', string='Τιμή ανά τ.μ.').parent.find_next('div')
        pricesper.append(priceper.text.strip())
    except AttributeError:
        pricesper.append('None')
    
    try:    
        size= soup2.find('h6', string='Εμβαδό').parent.find_next('div')
        sizes.append(size.text.strip())
    except AttributeError:
        sizes.append('None')
    
    try:
        kind= soup2.find('h6', string='Τύπος').parent.find_next('div')
        kinds.append(kind.text.strip())
    except AttributeError:
        kinds.append('None')
        
    try:
        num_of_room= soup2.find('h6', string='Υπνοδωμάτια').parent.find_next('div')
        num_of_rooms.append(num_of_room.text.strip())
    except AttributeError:
        num_of_rooms.append('None')
    
    try:
        num_of_bath= soup2.find('h6', string='Μπάνια').parent.find_next('div')
        num_of_baths.append(num_of_bath.text.strip())
    except AttributeError:
        num_of_baths.append('None')
    
    
    try:
        floor= soup2.find('h6', string='Όροφος').parent.find_next('div')
        floors.append(floor.text.strip())
    except AttributeError:
        floors.append('None')
    
    try:
        park= soup2.find('h6', string='Θέση στάθμευσης').parent.find_next('div')
        parking.append(park.text.strip())
    except AttributeError:
        parking.append('None')
    
    try:
        year_buitl= soup2.find('h6', string='Έτος κατασκευής').parent.find_next('div')
        year_builts.append(year_buitl.text.strip())
    except AttributeError:
        year_builts.append('None')
        
    try:
        last_updates= soup2.find('h6',string='Τελευταία ενημέρωση').parent.find_next('div')
        last_update.append(last_updates.text.strip())
    except AttributeError:
        last_update.append('None')
    
    try:
        areas= soup2.find('h6',string='Περιοχή').parent.find_next('div')
        area.append(areas.text.strip())
    except AttributeError:
        area.append('None')

In [ ]:
# Finally we append everything into a dataframe for further analysis and save it into a file that the user must change everytime
# a new area is being screaped.

In [ ]:
unique_info= pd.DataFrame(
                    {'prices':prices,
                     'priceper':pricesper,
                     'size':sizes,
                     'kind':kinds, 
                     'num_of_rooms':num_of_rooms,
                     'num_of_bath':num_of_baths,
                     'floor':floors,
                     'parking':parking,
                     'year_buitl':year_builts,
                     'last_update':last_update,
                     'area':area})    
    
unique_info.to_csv('spitogatos_center.csv')